# 将HDF5图像数据转换为视频

本notebook用于从RoboMind数据集的HDF5文件中提取RGB图像序列并生成视频文件。

## 功能特性

- ✅ 解码压缩的图像数据
- ✅ 支持多相机视角（chest, head）
- ✅ 生成单独相机视频
- ✅ 生成并排对比视频
- ✅ 添加信息覆盖层（帧号、时间戳等）
- ✅ 批量处理多个文件
- ✅ 进度条显示

## 使用方法

1. 配置参数（第2个cell）
2. 运行预览（第4个cell）
3. 生成视频（第5-7个cells）

## 1. 导入依赖

In [1]:
import h5py
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✅ 所有依赖已导入")
print(f"OpenCV版本: {cv2.__version__}")

✅ 所有依赖已导入
OpenCV版本: 4.12.0


## 2. 配置参数

In [ ]:
# ==================== 主要配置 ====================

# HDF5文件路径
H5_PATH = "/projects/p32958/chengxuan/data/robomind_fail/failure_data/135-place_cup_on_blue_bowl_with_left_hand/夹取前碰撞/1135137-2025_03_05_16_57_11/data/trajectory.hdf5"

# 输出目录
OUTPUT_DIR = "/projects/b1222/userdata/jianshu/chengxuan/saved/videos"

# 视频配置
VIDEO_CONFIG = {
    'fps': 30,                    # 帧率
    'codec': 'mp4v',              # 编码器: 'mp4v', 'avc1', 'H264', 'XVID'
    'quality': 95,                # 质量 (0-100, 仅对某些编码器有效)
    'resize': None,               # 调整大小 (width, height) 或 None 保持原始
    'add_frame_number': True,     # 是否添加帧号
    'add_timestamp': True,        # 是否添加时间戳
    'add_task_name': True,        # 是否添加任务名称
}

# 要处理的相机列表
CAMERAS = ['camera_chest', 'camera_head']

print("✅ 配置已加载")
print(f"输入文件: {H5_PATH}")
print(f"输出目录: {OUTPUT_DIR}")
print(f"帧率: {VIDEO_CONFIG['fps']} FPS")
print(f"编码器: {VIDEO_CONFIG['codec']}")

✅ 配置已加载
输入文件: /projects/p32958/chengxuan/data/robomind_fail/failure_data/135-place_cup_on_blue_bowl_with_left_hand/夹取前碰撞/1135137-2025_03_05_16_57_11/data/trajectory.hdf5
输出目录: /projects/p32958/chengxuan/data/robomind_fail/failure_data/135-place_cup_on_blue_bowl_with_left_hand/夹取前碰撞/1135137-2025_03_05_16_57_11/videos
帧率: 30 FPS
编码器: mp4v


## 3. 工具函数

In [ ]:
class HDF5VideoConverter:
    """HDF5图像数据到视频的转换器"""
    
    def __init__(self, h5_path: str, output_dir: str, config: dict):
        self.h5_path = Path(h5_path)
        self.output_dir = Path(output_dir)
        self.config = config
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # 验证文件存在
        if not self.h5_path.exists():
            raise FileNotFoundError(f"HDF5文件不存在: {self.h5_path}")
        
        # 加载基本信息
        with h5py.File(self.h5_path, 'r') as f:
            self.is_compressed = f.attrs.get('compress', True)
            self.task_name = f['language_instruction'][()].decode('utf-8') if 'language_instruction' in f else 'Unknown Task'
            
            # 检查可用的相机
            self.available_cameras = []
            if 'observations/rgb_images' in f:
                self.available_cameras = list(f['observations/rgb_images'].keys())
            
            # 获取时间序列
            self.time_seq = f['sim_time_seq'][:] if 'sim_time_seq' in f else None
        
        print(f"✅ 转换器初始化成功")
        print(f"   任务: {self.task_name}")
        print(f"   压缩: {self.is_compressed}")
        print(f"   可用相机: {self.available_cameras}")
    
    def decode_image(self, compressed_data: np.ndarray) -> Optional[np.ndarray]:
        """解码压缩的图像数据"""
        if compressed_data is None or len(compressed_data) == 0:
            return None
        
        try:
            # 尝试使用cv2解码
            img = cv2.imdecode(compressed_data, cv2.IMREAD_COLOR)
            
            if img is None:
                # 尝试作为原始字节解析
                img = np.frombuffer(compressed_data, dtype=np.uint8)
                
                # 根据大小推断分辨率
                if img.size == 2764800:  # 720 * 1280 * 3
                    img = img.reshape(720, 1280, 3)
                elif img.size == 921600:  # 480 * 640 * 3
                    img = img.reshape(480, 640, 3)
                elif img.size == 1228800:  # 640 * 640 * 3
                    img = img.reshape(640, 640, 3)
                else:
                    return None
            
            # BGR转RGB
            if len(img.shape) == 3 and img.shape[2] == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            return img
            
        except Exception as e:
            print(f"解码失败: {e}")
            return None
    
    def get_frames(self, camera_name: str) -> Tuple[int, callable]:
        """获取指定相机的帧数和帧获取函数"""
        with h5py.File(self.h5_path, 'r') as f:
            camera_data = f['observations/rgb_images'][camera_name]
            num_frames = len(camera_data)
            
            def get_frame(idx):
                with h5py.File(self.h5_path, 'r') as f_inner:
                    frame_data = f_inner['observations/rgb_images'][camera_name][idx]
                    if self.is_compressed:
                        return self.decode_image(frame_data)
                    else:
                        img = frame_data
                        if len(img.shape) == 3 and img.shape[2] == 3:
                            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        return img
            
            return num_frames, get_frame
    
    def add_info_overlay(self, img: np.ndarray, frame_idx: int, 
                        camera_name: str = "") -> np.ndarray:
        """在图像上添加信息覆盖层"""
        img_with_info = img.copy()
        
        # 转换为BGR用于OpenCV绘制
        img_with_info = cv2.cvtColor(img_with_info, cv2.COLOR_RGB2BGR)
        
        # 配置文本样式
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.6
        thickness = 2
        color = (255, 255, 255)  # 白色
        bg_color = (0, 0, 0)      # 黑色背景
        
        y_offset = 30
        
        # 添加任务名称
        if self.config.get('add_task_name', True):
            text = f"Task: {self.task_name}"
            (w, h), _ = cv2.getTextSize(text, font, font_scale, thickness)
            cv2.rectangle(img_with_info, (10, y_offset - h - 5), 
                         (15 + w, y_offset + 5), bg_color, -1)
            cv2.putText(img_with_info, text, (10, y_offset), 
                       font, font_scale, color, thickness)
            y_offset += 35
        
        # 添加相机名称
        if camera_name:
            text = f"Camera: {camera_name}"
            (w, h), _ = cv2.getTextSize(text, font, font_scale, thickness)
            cv2.rectangle(img_with_info, (10, y_offset - h - 5), 
                         (15 + w, y_offset + 5), bg_color, -1)
            cv2.putText(img_with_info, text, (10, y_offset), 
                       font, font_scale, color, thickness)
            y_offset += 35
        
        # 添加帧号
        if self.config.get('add_frame_number', True):
            text = f"Frame: {frame_idx}"
            (w, h), _ = cv2.getTextSize(text, font, font_scale, thickness)
            cv2.rectangle(img_with_info, (10, y_offset - h - 5), 
                         (15 + w, y_offset + 5), bg_color, -1)
            cv2.putText(img_with_info, text, (10, y_offset), 
                       font, font_scale, color, thickness)
            y_offset += 35
        
        # 添加时间戳
        if self.config.get('add_timestamp', True) and self.time_seq is not None:
            if frame_idx < len(self.time_seq):
                text = f"Time: {self.time_seq[frame_idx]:.3f}s"
                (w, h), _ = cv2.getTextSize(text, font, font_scale, thickness)
                cv2.rectangle(img_with_info, (10, y_offset - h - 5), 
                             (15 + w, y_offset + 5), bg_color, -1)
                cv2.putText(img_with_info, text, (10, y_offset), 
                           font, font_scale, color, thickness)
        
        # 转换回RGB
        img_with_info = cv2.cvtColor(img_with_info, cv2.COLOR_BGR2RGB)
        return img_with_info
    
    def create_single_camera_video(self, camera_name: str, 
                                   add_info: bool = False) -> Path:
        """为单个相机创建视频"""
        print(f"\n🎬 开始生成 {camera_name} 视频...")
        
        if camera_name not in self.available_cameras:
            raise ValueError(f"相机 {camera_name} 不可用. 可用相机: {self.available_cameras}")
        
        # 获取帧信息
        num_frames, get_frame = self.get_frames(camera_name)
        print(f"   总帧数: {num_frames}")
        
        # 获取第一帧以确定分辨率
        first_frame = get_frame(0)
        if first_frame is None:
            raise RuntimeError("无法解码第一帧")
        
        height, width = first_frame.shape[:2]
        
        # 应用resize配置
        if self.config.get('resize'):
            width, height = self.config['resize']
        
        print(f"   分辨率: {width}x{height}")
        
        # 创建视频写入器
        suffix = "_with_info" if add_info else ""
        output_path = self.output_dir / f"{camera_name}{suffix}.mp4"
        
        fourcc = cv2.VideoWriter_fourcc(*self.config['codec'])
        writer = cv2.VideoWriter(
            str(output_path),
            fourcc,
            self.config['fps'],
            (width, height)
        )
        
        if not writer.isOpened():
            raise RuntimeError(f"无法创建视频写入器. 编码器: {self.config['codec']}")
        
        # 逐帧写入
        for frame_idx in tqdm(range(num_frames), desc=f"  处理 {camera_name}"):
            frame = get_frame(frame_idx)
            
            if frame is None:
                print(f"\n⚠️  警告: 帧 {frame_idx} 解码失败，跳过")
                continue
            
            # 添加信息覆盖
            if add_info:
                frame = self.add_info_overlay(frame, frame_idx, camera_name)
            
            # Resize if needed
            if self.config.get('resize'):
                frame = cv2.resize(frame, (width, height))
            
            # 转换RGB到BGR for OpenCV
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            writer.write(frame_bgr)
        
        writer.release()
        print(f"✅ 视频已保存: {output_path}")
        
        return output_path
    
    def create_side_by_side_video(self, cameras: List[str], 
                                  add_info: bool = False) -> Path:
        """创建并排对比视频"""
        print(f"\n🎬 开始生成并排对比视频...")
        print(f"   相机: {cameras}")
        
        # 验证相机
        for cam in cameras:
            if cam not in self.available_cameras:
                raise ValueError(f"相机 {cam} 不可用")
        
        # 获取所有相机的帧信息
        frame_getters = []
        num_frames_list = []
        
        for cam in cameras:
            num_frames, get_frame = self.get_frames(cam)
            frame_getters.append(get_frame)
            num_frames_list.append(num_frames)
        
        # 使用最小帧数
        num_frames = min(num_frames_list)
        print(f"   总帧数: {num_frames}")
        
        # 获取第一帧确定分辨率
        first_frames = [getter(0) for getter in frame_getters]
        
        if any(f is None for f in first_frames):
            raise RuntimeError("无法解码第一帧")
        
        # 获取最大高度，总宽度
        heights = [f.shape[0] for f in first_frames]
        widths = [f.shape[1] for f in first_frames]
        
        max_height = max(heights)
        total_width = sum(widths)
        
        print(f"   分辨率: {total_width}x{max_height}")
        
        # 创建视频写入器
        suffix = "_with_info" if add_info else ""
        output_path = self.output_dir / f"combined_side_by_side{suffix}.mp4"
        
        fourcc = cv2.VideoWriter_fourcc(*self.config['codec'])
        writer = cv2.VideoWriter(
            str(output_path),
            fourcc,
            self.config['fps'],
            (total_width, max_height)
        )
        
        if not writer.isOpened():
            raise RuntimeError(f"无法创建视频写入器")
        
        # 逐帧写入
        for frame_idx in tqdm(range(num_frames), desc="  处理并排视频"):
            frames = [getter(frame_idx) for getter in frame_getters]
            
            # 跳过任何None帧
            if any(f is None for f in frames):
                print(f"\n⚠️  警告: 帧 {frame_idx} 解码失败，跳过")
                continue
            
            # 添加信息覆盖
            if add_info:
                frames = [self.add_info_overlay(f, frame_idx, cam) 
                         for f, cam in zip(frames, cameras)]
            
            # 调整所有帧到相同高度
            resized_frames = []
            for frame in frames:
                if frame.shape[0] != max_height:
                    aspect_ratio = frame.shape[1] / frame.shape[0]
                    new_width = int(max_height * aspect_ratio)
                    frame = cv2.resize(frame, (new_width, max_height))
                resized_frames.append(frame)
            
            # 水平拼接
            combined = np.hstack(resized_frames)
            
            # 转换RGB到BGR
            combined_bgr = cv2.cvtColor(combined, cv2.COLOR_RGB2BGR)
            writer.write(combined_bgr)
        
        writer.release()
        print(f"✅ 并排视频已保存: {output_path}")
        
        return output_path
    
    def preview_frames(self, camera_name: str, frame_indices: List[int] = None):
        """预览指定相机的关键帧"""
        print(f"\n🔍 预览 {camera_name} 的关键帧...")
        
        num_frames, get_frame = self.get_frames(camera_name)
        
        # 默认预览首帧、中间帧、末帧
        if frame_indices is None:
            frame_indices = [0, num_frames // 2, num_frames - 1]
        
        fig, axes = plt.subplots(1, len(frame_indices), figsize=(5 * len(frame_indices), 5))
        if len(frame_indices) == 1:
            axes = [axes]
        
        for idx, frame_idx in enumerate(frame_indices):
            frame = get_frame(frame_idx)
            
            if frame is not None:
                axes[idx].imshow(frame)
                axes[idx].set_title(f"{camera_name}\nFrame {frame_idx}/{num_frames-1}")
                axes[idx].axis('off')
            else:
                axes[idx].text(0.5, 0.5, 'Decode Failed', 
                             ha='center', va='center', fontsize=16)
                axes[idx].set_title(f"Frame {frame_idx} (failed)")
        
        plt.tight_layout()
        plt.show()

# 创建转换器实例
converter = HDF5VideoConverter(H5_PATH, OUTPUT_DIR, VIDEO_CONFIG)

print("\n✅ 转换器类已定义并初始化")

## 4. 预览关键帧

在生成视频之前，先预览几帧以确认图像解码正常

In [ ]:
# 预览camera_chest的关键帧
converter.preview_frames('camera_chest')

In [ ]:
# 预览camera_head的关键帧
converter.preview_frames('camera_head')

## 5. 生成单相机视频

In [ ]:
# 生成camera_chest视频（不带信息覆盖）
video_path_chest = converter.create_single_camera_video('camera_chest', add_info=False)

In [ ]:
# 生成camera_head视频（不带信息覆盖）
video_path_head = converter.create_single_camera_video('camera_head', add_info=False)

## 6. 生成带信息覆盖的视频

这将在视频上叠加帧号、时间戳和任务名称等信息

In [ ]:
# 生成camera_chest视频（带信息覆盖）
video_path_chest_info = converter.create_single_camera_video('camera_chest', add_info=True)

In [ ]:
# 生成camera_head视频（带信息覆盖）
video_path_head_info = converter.create_single_camera_video('camera_head', add_info=True)

## 7. 生成并排对比视频

In [ ]:
# 生成并排视频（不带信息覆盖）
side_by_side_path = converter.create_side_by_side_video(
    cameras=['camera_chest', 'camera_head'],
    add_info=False
)

In [ ]:
# 生成并排视频（带信息覆盖）
side_by_side_path_info = converter.create_side_by_side_video(
    cameras=['camera_chest', 'camera_head'],
    add_info=True
)

## 8. 批量处理多个HDF5文件

如果您需要处理同一目录下的多个HDF5文件，可以使用以下代码

In [ ]:
def batch_process_h5_files(base_dir: str, pattern: str = "**/trajectory.hdf5", 
                          video_config: dict = None):
    """
    批量处理目录下的所有HDF5文件
    
    Args:
        base_dir: 基础目录
        pattern: 文件匹配模式
        video_config: 视频配置
    """
    base_path = Path(base_dir)
    h5_files = list(base_path.rglob(pattern))
    
    print(f"🔍 找到 {len(h5_files)} 个HDF5文件\n")
    
    if video_config is None:
        video_config = VIDEO_CONFIG
    
    results = []
    
    for i, h5_file in enumerate(h5_files, 1):
        print(f"\n{'='*80}")
        print(f"处理 [{i}/{len(h5_files)}]: {h5_file.name}")
        print(f"{'='*80}")
        
        try:
            # 创建输出目录（与HDF5文件同级的videos目录）
            output_dir = h5_file.parent.parent / 'videos'
            
            # 创建转换器
            converter = HDF5VideoConverter(str(h5_file), str(output_dir), video_config)
            
            # 生成并排视频
            video_path = converter.create_side_by_side_video(
                cameras=['camera_chest', 'camera_head'],
                add_info=True
            )
            
            results.append({
                'h5_file': str(h5_file),
                'video_path': str(video_path),
                'status': 'success'
            })
            
        except Exception as e:
            print(f"❌ 处理失败: {e}")
            results.append({
                'h5_file': str(h5_file),
                'video_path': None,
                'status': 'failed',
                'error': str(e)
            })
    
    # 打印汇总
    print(f"\n{'='*80}")
    print("批量处理完成")
    print(f"{'='*80}")
    
    success_count = sum(1 for r in results if r['status'] == 'success')
    failed_count = len(results) - success_count
    
    print(f"\n✅ 成功: {success_count}")
    print(f"❌ 失败: {failed_count}")
    
    if failed_count > 0:
        print("\n失败列表:")
        for r in results:
            if r['status'] == 'failed':
                print(f"  - {r['h5_file']}")
                print(f"    错误: {r['error']}")
    
    return results

# 示例：批量处理
# base_directory = "/projects/p32958/chengxuan/data/robomind_fail/failure_data"
# results = batch_process_h5_files(base_directory)

## 9. 视频信息查看

查看生成的视频信息

In [ ]:
def get_video_info(video_path: str) -> dict:
    """获取视频文件信息"""
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        return {"error": "无法打开视频文件"}
    
    info = {
        'path': video_path,
        'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        'fps': cap.get(cv2.CAP_PROP_FPS),
        'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        'codec': int(cap.get(cv2.CAP_PROP_FOURCC)),
    }
    
    # 计算时长
    if info['fps'] > 0:
        info['duration'] = info['frame_count'] / info['fps']
    else:
        info['duration'] = 0
    
    # 文件大小
    file_size = Path(video_path).stat().st_size
    info['file_size_mb'] = file_size / 1024 / 1024
    
    cap.release()
    
    return info

def print_video_info(video_path: str):
    """打印视频信息"""
    info = get_video_info(video_path)
    
    if 'error' in info:
        print(f"❌ {info['error']}")
        return
    
    print(f"\n📹 视频信息: {Path(video_path).name}")
    print(f"   分辨率: {info['width']}x{info['height']}")
    print(f"   帧率: {info['fps']:.2f} FPS")
    print(f"   总帧数: {info['frame_count']}")
    print(f"   时长: {info['duration']:.2f} 秒")
    print(f"   文件大小: {info['file_size_mb']:.2f} MB")

# 查看生成的视频信息
print("\n" + "="*80)
print("生成的视频信息")
print("="*80)

# 列出输出目录中的所有视频
output_path = Path(OUTPUT_DIR)
if output_path.exists():
    video_files = sorted(output_path.glob("*.mp4"))
    
    if video_files:
        for video_file in video_files:
            print_video_info(str(video_file))
    else:
        print("\n⚠️  输出目录中没有找到视频文件")
else:
    print(f"\n⚠️  输出目录不存在: {output_path}")

## 10. 快速生成（推荐）

如果您只想快速生成带信息的并排视频，运行这个cell即可

In [ ]:
# 一键生成带信息的并排视频
print("🚀 快速生成模式：生成带信息的并排视频\n")

# 重新创建转换器（如果需要）
converter = HDF5VideoConverter(H5_PATH, OUTPUT_DIR, VIDEO_CONFIG)

# 生成视频
video_path = converter.create_side_by_side_video(
    cameras=['camera_chest', 'camera_head'],
    add_info=True
)

# 显示视频信息
print_video_info(str(video_path))

print(f"\n✅ 完成！视频已保存到: {video_path}")